In [15]:
# notebook for streamlit dev

# library imports

import streamlit as st
import pandas as pd
import numpy as np
from pathlib import Path
from os.path import realpath,dirname
import os
import csv
import matplotlib.pyplot as plt

from typing import List,Any

freq_option = 'Month'

#freq_option = st.selectbox('Date Level Selector',['Year','Quarter','Month','Week','Day'])
freq_dict = {
     'Year':'y'
    ,'Month':'m'
    ,'Quarter':'q'
    ,'Week':'w'
    ,'Day':'d'
}


freq = freq_dict.get(freq_option)


SHEET_TYPES = {
     'date': 'datetime64[ns]'
    ,'category':str
    ,'type':str
    ,'amount':np.dtype('float32')
    ,'description':str
    ,'amount_cents':np.dtype('int32')
}

SHEET_COLUMNS = list(SHEET_TYPES.keys())


CATEGORIES = [

     'Transportation'
    ,'Daily Living'
    ,'Entertainment'
    ,'Subscriptions'
    ,'Home Expenses'
    ,'Health'
    ,'Savings'
    ,'Misc'
    ,'Obligations'
    ,'Charity And Gifts'
]

def transform_g_sheet_array(array:List[List[Any]])->pd.DataFrame:
    if array[0] != SHEET_COLUMNS:
        raise ValueError("Invalid sheet passed.")
    #TODO: add additional arguements
    df = pd.DataFrame(data=array[1:],columns=array[0])

    for col, dtype in SHEET_TYPES.items():
        df[col] = df[col].astype(dtype)

    return df



def make_dataframe()->pd.DataFrame:
    #root_path = Path(dirname(realpath(__file__)))
    root_path = Path(realpath(''))
    csv_folder_path = root_path/'..'/'csvs'

    for root,dirs,files in os.walk(csv_folder_path):
        dfs = list()
        for file in files:
            if file.endswith('.csv'):
                with open(csv_folder_path/file,mode='r') as f:
                    reader = csv.reader(f)
                    dfs.append(transform_g_sheet_array(array=[row for row in reader]))

    dfs = pd.concat(dfs,axis=0,sort=True).reset_index(drop=True)
    return dfs

def process_dataframe(df:pd.DataFrame)->pd.DataFrame:
    all_dates = pd.date_range(start= df.date.min(),end=df.date.max(),freq='D')
    index = pd.MultiIndex.from_product([all_dates,CATEGORIES],names=['date','category'])

    full_df = pd.DataFrame(index=index).reset_index(drop=False)

    new_df = full_df.merge(df,left_on=['date','category'],right_on=['date','category'],how='left')
    new_df.amount_cents = new_df.amount_cents.replace(np.nan,0)
    new_df.amount = new_df.amount.replace(np.nan,0)

    new_df = new_df.groupby(['date','category'],as_index=False)[['amount_cents']].sum()
    new_df['amount_cents']  = new_df.amount_cents/100
    new_df = new_df.set_index(keys="date")


    plot_df = new_df.groupby([pd.Grouper(freq=freq),'category']).sum().reset_index(drop=False)
    pivot_df = plot_df.pivot(index='date',columns='category',values='amount_cents')

    return pivot_df



In [16]:
df = make_dataframe()
pivot_df = process_dataframe(df=df)

total = pivot_df.sum()
total.name = 'Category'

In [20]:
pivot_df.T.sum().to_frame().rename({0:'Monthly Total'},axis=1)

,Monthly Total
date,
2022-06-30,2635.90
2022-07-31,2851.52
2022-08-31,47.11
